In [1]:
#Import numpy and pandas
import pandas as pd 
import numpy as np


In [2]:
#read data(I got this dataset from kaggle)
data=pd.read_csv("E:LearningAndExploring/Data Science/Data/SpamClassifier-master/smsspamcollection/SMSSpamCollection.csv",sep="\t")

In [5]:
#naming columns 
data.columns=['class','email']

In [6]:
#checking first five rows of data
data.head()

,class,email
0,ham,Ok lar... Joking wif u oni...
1,spam,Free entry in 2 a wkly comp to win FA Cup fina...
2,ham,U dun say so early hor... U c already then say...
3,ham,"Nah I don't think he goes to usf, he lives aro..."
4,spam,FreeMsg Hey there darling it's been 3 week's n...


In [7]:
#encoding class columns.created label columns ham as 0 and spam as 1
data['label']=data['class'].map({'ham':0,'spam':1})

In [8]:
data.head()

,class,email,label
0,ham,Ok lar... Joking wif u oni...,0
1,spam,Free entry in 2 a wkly comp to win FA Cup fina...,1
2,ham,U dun say so early hor... U c already then say...,0
3,ham,"Nah I don't think he goes to usf, he lives aro...",0
4,spam,FreeMsg Hey there darling it's been 3 week's n...,1


In [29]:
data.iloc[3].email

"Nah I don't think he goes to usf, he lives around here though"

In [9]:
#differentiating dependent and independent variables

x=data.email
y=data.label

In [10]:
x.shape,y.shape

((5570,), (5570,))

In [11]:
#import necessary libraries for text preprocessing and
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import re

In [12]:
lemmatizer=WordNetLemmatizer()

In [13]:
#Lemmatize and remove stopwords from text emails
sentences=[]
for i in range(len(x)):
    sent=x.iloc[i].lower()
    words=nltk.word_tokenize(sent)
    words=[lemmatizer.lemmatize(word) for word in words if word not in set(stopwords.words('english'))]
    sent=' '.join(words)
    sentences.append(sent)

In [14]:
len(sentences)

5570

In [18]:
#we will convert above sentences into vectors using BagOfWord Technique
from sklearn.feature_extraction.text import CountVectorizer

In [19]:
cv=CountVectorizer(max_features=4000)

In [20]:
#transforming independent variable into vectors
x=cv.fit_transform(sentences).toarray()

In [21]:
x.shape

(5570, 4000)

In [24]:
from sklearn.model_selection import train_test_split

In [25]:
#splitting training and validation data
X_train, X_test, y_train, y_test = train_test_split(x,y,test_size=0.2,random_state=42)

In [22]:
#will build our model using Multinomial Naive Bayes
from sklearn.naive_bayes import MultinomialNB

In [23]:
model=MultinomialNB()

In [26]:
#fitting to our model for training 
model.fit(X_train,y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [27]:
#evaluating model
model.score(X_test,y_test)

0.981149012567325

In [28]:
#testing function 
def predict_spam(text):
    sent=text.lower()
    words=nltk.word_tokenize(sent)
    words=[lemmatizer.lemmatize(word) for word in words if word not in set(stopwords.words('english'))]
    sent=' '.join(words)
    temp=cv.transform([sent]).toarray()
    answer=model.predict(temp)
    return answer

In [33]:
#testing our model using predict_spam function we created above
predict_spam("free offer specially for you")

array([1], dtype=int64)

In [34]:
import pickle

In [35]:
#oickling model so that we can load in our flask server and use it without training it again
pickle.dump(model,open("model.pkl",'wb'))

In [36]:
#we will CountVectorizer object too in flask server hence pickling that too
pickle.dump(cv,open("cv.pkl",'wb'))